In [25]:
from ortools.sat.python import cp_model
import collections
import numpy as np

# Problema de Optimización Lineal Entera

Nuestro objetivo es crear un modelo que sea capaz de asignar tareas a trabajadores indicando que herramientas utilizarán.  

Para realizar esto consideraremos que un óptimo local aceptable será que la última tarea a realizar sea lo antes posible.  

Por otro lado cabe destacar que para un conjunto de tareas $\{{t_{1},\ldots,t_{n} }\}$ cada tarea $t_{i}$ estará asignada a un operador en un intervalo de tiempo $[s_{i,j}, e_{i,j}]$ donde s y e indican el inicio y termino de de la tarea i y el trabajador j.  

Para un par de tareas $t_{i} \: t_{ii}$  para el mismo operador se debe cumplir que $e_{i,j}<s_{ii,j}$ ó $e_{ii,j} < s_{i,j}$.  

Del mismo modo podemos definir el solapamiento para las herramientas que se utilizaran para cada tarea.  

Con respecto a las variables a utilizar en el problema utilizaremos las siguientes

- T<sub>i</sub>  donde i hace referencia a una tarea i $\in$ al conjunto total de Tareas  
- O<sub>i</sub>  donde i hace referencia al operador i $\in$ al conjunto total de Operadores  
- A<sub>i, j</sub>  este valor es 1 si la tarea i se asigna al operador j, es 0 sino  
- D<sub>i, j</sub>  Duración del trabajo i para el operador j  
- C<sub>i, j</sub>  1 si el operador j esta habilitado para realizar i 0 \~  
- H<sub>k,i</sub> 1 si la tarea i se puede desarrollar con la herramienta k 0 \~



\begin{aligned}
    &Z= \min(\max (\vec{T}) )_{i \in [T_{1}, \ldots, T_{n}] }\\
    & \text{sujeto a} \\
    &  \sum_{j=1}^{M}A_{I,j}*C_{I,j}=1 \; \forall \: j \in O\\
    &  \sum_{i=1}^{N} \sum_{j=1}^{M}A_{i,j}=N \; \forall \: j \in O \; y \; \forall \: i \in T \\
     &\sum_{i=1}^{N} t_i = N \;  \forall \: t_{i} \in T \\
    &  
\end{aligned}








In [1]:
tareas = {
    "tarea_1": {'disciplina':'electrica', "duracion": 5},
    "tarea_2": {'disciplina':'electrica',  "duracion": 2},
    "tarea_3": {'disciplina':'construccion',  "duracion": 1},
    "tarea_4": {'disciplina':'electrica',  "duracion": 3},
    "tarea_5": {'disciplina':'electrica',  "duracion": 6},
    "tarea_6": {'disciplina':'construccion',  "duracion": 9},
    "tarea_7": {'disciplina':'construccion',  "duracion": 2},
    "tarea_8": {'disciplina':'construccion',  "duracion": 4},
    "tarea_9": {'disciplina':'electrica',  "duracion": 1},
    "tarea_10": {'disciplina':'construccion', "duracion": 7},
}
          
operadores = {
    "BlancoEncalada":{"disciplina":"electrica","eficiencia": 1},
    "AnibalPinto":{"disciplina":"electrica","eficiencia": 2},
    "FedericoErrazuriz":{"disciplina":"construccion","eficiencia": 3},
    "ManuelMontt":{"disciplina":"construccion","eficiencia": 1},
}



In [166]:
herramientas ={
    "1401": {"disciplina":'electrica'},
    "1501": {"disciplina":'construccion'},
    "1402": {"disciplina":'electrica'},
}
entregas = {
    "tarea_1": 15,
    "tarea_2": 200,
    "tarea_3": 16,
    "tarea_4": 400,
    "tarea_5": 400,
    "tarea_6": 400,
    "tarea_7": 400,
    "tarea_8": 400,
    "tarea_9": 400,
    "tarea_10":400,
}

In [167]:
duracion = {'electrica': {'min':np.inf,'max':0}, 'construccion': {'min':np.inf,'max':0}}
for key,dic in tareas.items():
    for operador in operadores:
        if operadores[operador]['disciplina']==dic['disciplina']:
            duracion[dic['disciplina']]['max'] = max(duracion[dic['disciplina']]['max'], operadores[operador]['eficiencia'])
            duracion[dic['disciplina']]['min'] = min(duracion[dic['disciplina']]['max'], operadores[operador]['eficiencia'])

In [168]:
model = cp_model.CpModel()

horizon = sum([dic['duracion']*duracion[dic['disciplina']]['max'] for tarea, dic in tareas.items()])

presencias = {}  
termino_tareas = []
inicio_tareas = []
solutions = {}
intervalos_operadores = collections.defaultdict(list)

for tarea in tareas:
    sufijo = f"_{tarea}"
    start = model.new_int_var(0, horizon, "start" + sufijo)
    minima_duracion = tareas[tarea]['duracion']*duracion[tareas[tarea]['disciplina']]['min']
    maxima_duracion = tareas[tarea]['duracion']*duracion[tareas[tarea]['disciplina']]['max']
    duration = model.new_int_var(minima_duracion, maxima_duracion, "duration" + sufijo)
    end = model.new_int_var(0, horizon, "end" + sufijo)
    interval = model.new_interval_var(start, duration, end, "interval" + sufijo)
    p_presences = []
    for operador in operadores:
        if tareas[tarea]['disciplina'] == operadores[operador]['disciplina']:
            alternativa_sufijo = f"_j{tarea}_o{operador}"
            p_presence = model.new_bool_var("presence" + alternativa_sufijo)
            p_start = model.new_int_var(0, horizon, "start" + alternativa_sufijo)
            p_duration = tareas[tarea]['duracion'] * operadores[operador]['eficiencia']
            p_end = model.new_int_var(0, horizon, "end" + alternativa_sufijo)
            p_interval = model.new_optional_interval_var(p_start, p_duration, p_end, p_presence, "interval" + alternativa_sufijo)
            p_presences.append(p_presence)

            model.add(start == p_start).only_enforce_if(p_presence)
            model.add(duration == p_duration).only_enforce_if(p_presence)
            model.add(end == p_end).only_enforce_if(p_presence)

            presencias[(tarea, operador)] = p_presence
            operator_interval = model.new_interval_var(start, duration, end, "interval" + sufijo)
            intervalos_operadores[operador].append(p_interval)
            
            

    model.add_exactly_one(p_presences) #model.Add(sum(array_of_vars) == 1)
    inicio_tareas.append(start)
    termino_tareas.append(end)
    solutions[tarea] = {'inicio': start, 'termino': end}
    
for operador in operadores:
    model.add_no_overlap(intervalos_operadores[operador])


In [170]:
makespan = model.new_int_var(0, horizon, "makespan")
model.add_max_equality(makespan, termino_tareas)
model.minimize(makespan)

solver = cp_model.CpSolver()
status = solver.Solve(model)

out = []
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # Mostrar makespan
    print(f"Tiempo total mínimo (makespan): {solver.Value(makespan)}\n")
    
    # Mostrar inicio y fin de tareas
    for key in presencias:
        if solver.Value(presencias[key]):
            out.append([key[0], key[1], solver.Value(solutions[key[0]]['inicio']), solver.Value(solutions[key[0]]['termino'])])

else:
    print("No se encontró solución.")

Tiempo total mínimo (makespan): 34



In [192]:
solucion = {i:[] for i in list(set([values[1] for values in out]))}
for w in out:
    solucion[w[1]].append([w[2], w[3]])
    
for k, l in solucion.items():
    print(k, end=': ')
    inicio = [i[0] for i in l]
    termino = [i[1] for i in l]
    inicio.sort()
    termino.sort()
    for n, i in enumerate(inicio):
        print(i, termino[n], end="....")
    print()

AnibalPinto: 0 12....12 22....22 28....28 32....32 34....
ManuelMontt: 0 1....1 5....5 12....
FedericoErrazuriz: 0 6....6 33....
